In [42]:
#Parsing STiX 1.x Data and creating a common format
import xml.etree.ElementTree as ET
tree = ET.parse("./dataset/stix1_sample.xml")
modified_dict = {
    "stix_version": "",
    "entity_type": "",
    "indicator_type": "",
    "description": "",
    "timestamp":"",
    "values": None
}
root = tree.getroot()
version = root.get("version")
modified_dict["stix_version"]=version
ns = {
    "stix": "http://docs.oasis-open.org/cti/ns/stix/core-1",
    "indicator": "http://docs.oasis-open.org/cti/ns/stix/indicator-1",
    "URIObject": "http://docs.oasis-open.org/cti/ns/cybox/objects/uri-2"
}
entity_type_array = ["Indicator", "Incident", "TTP", "Campaign", "ThreatActor"]
def strip_ns(tag):
    return tag.split("}")[-1]
for elem in root.iter():
    clean = strip_ns(elem.tag)
    if clean in entity_type_array:
        modified_dict["entity_type"]=clean
mod_array = [".//indicator:Type", ".//indicator:Description", ".//URIObject:Value"]
new = [root.find(i, ns).text for i in mod_array]
modified_dict["indicator_type"]=new[0]
modified_dict["description"]=new[1]
modified_dict["values"]=new[2].split("##comma##")
indicator = root.find(".//stix:Indicator", ns)
modified_dict["timestamp"] = indicator.get("timestamp")
print(modified_dict)

{'stix_version': 'stix-1.2.1', 'entity_type': 'Indicator', 'indicator_type': 'URL Watchlist', 'description': 'Sample URL Indicator for this watchlist', 'timestamp': '2015-05-15T09:00:00.000000Z', 'values': ['http://example.com/foo/malicious1.html', 'http://example.com/foo/malicious2.html', 'http://example.com/foo/malicious3.html']}


In [40]:
#STiX 2.x Parsing
import json
with open("./dataset/stix20_sample.json") as f:
    item = json.load(f)
#print(len(item),"\n")
arr_of_data = []
def version_definition(data):
    try:
        item_out = list(data.keys())
        item_in = list(data["objects"][0].keys())
        merged_list = item_in + item_out
        print(merged_list)
        for indice in range(len(data["objects"])):
            modified_dict_20 = {
                "stix_version": "",
                "entity_type": "",
                "indicator_type": "",
                "description": "",
                "timestamp":"",
                "values": None
            }
            if "stix_version" in merged_list or "spec_version" in merged_list or "version" in merged_list:
                modified_dict_20["stix_version"] = 2.1
            else:
                modified_dict_20["stix_version"] = 2.0
            modified_dict_20["entity_type"] = item["objects"][indice]["type"]
            modified_dict_20["description"] = item["objects"][indice]["description"]
            modified_dict_20["timestamp"] = item["objects"][indice]["created"]
            modified_dict_20["indicator_type"] = item["objects"][indice]["labels"]
            object = item["objects"][indice]
            name = [object.get("name")] || ""
            alias = object.get("x_mitre_aliases") || []
            er = str(object.get("external_references")) || []
            merged_values = name + alias + er
            modified_dict_20["values"] = merged_values
            arr_of_data.append(modified_dict_20)
    except Exception as e:
        print("Error accessing STiX data!", e)
version_definition(item)

['type', 'id', 'created', 'created_by_ref', 'revoked', 'external_references', 'object_marking_refs', 'modified', 'name', 'description', 'x_mitre_modified_by_ref', 'x_mitre_platforms', 'x_mitre_deprecated', 'x_mitre_domains', 'x_mitre_version', 'x_mitre_attack_spec_version', 'x_mitre_aliases', 'labels', 'type', 'id', 'objects']
